# DEEP DETERMINISTIC POLICY GRADIENTS

Before we dive into this, lets just walk thorugh some basics
Polcy gradients methods are a new class of Reinforcement Learning algorithms

In [ ]:
#Put a better intro here - maybe add a brief intro about rl in general- 
#wjhat a policy is, what state, actions and reward are
#and we can also say this is helpful reading before continuing to policy gradients

# 1. Policy Gradients

## 1.1 Definition
Policy gradient methods learn a parameterized policy that can select actions without consulting a value function. They model and optimize the policy directly.
$$\pi(a|s, \theta)=Pr\{A_{t} = a\ | S_{t} = s, \theta_{t} = \theta\}$$

This equation denotes the probability that action a is taken at time t given that the environment is in state s at time t with parameter $\theta$.

An example of a policy:
\begin{equation*}
\pi(a|s, \theta) = \frac{e^{h(s, a, \theta)}} {\sum_{b}{e^{h(s, b, \theta)}}}
\end{equation*}

Reminds you off the softmax equation we so often see in machine learning doesn't it?
We call this kind of policy paramterization as *softmax in action preferences*.
Just like in machine learnning we would have $h(s, a, \theta) = \theta^T x(s, a)$ where $x(s, a)$ is a feature vector and $\theta$ are the weights.

Now that we covered the **Policy** portion, let's move on to the **Gradient** portion.

In [10]:
#Not sure how the feature vector is computed - will need to refer to chapter 9

## 1.2 Policy Gradient Theorem
Let's define our expected reward. We represent the total reward for a given trajectory $\tau$ as $r(\tau)$.
$$J(\theta) = \mathbb{E_{\pi}}[r(\tau)]$$
As we'vew seen before the expected reward is the value of the start state $s_{0}$ under a policy $\pi_{\theta}$.\
Therefore we have:
$$J(\theta) = v_{\pi_{\theta}}(s_{0}) = \mathbb{E_{\pi}}[r(\tau)]$$

The equations look pretty cool, but what next? 
We can derive some intuition from the loss functions used in machine learning. A loss function is defined with respect to the parameters $\theta$ and we use **gradient descent** to find the paramters $\theta$ that minimize the loss. 

$$\theta_{t+1} = \theta_{t} - \alpha \nabla L(\theta_{t})$$

In Reinforcement Learning however we want maximize the expected reward, so what do we do? Well pretty simple we go up instead of down, i.e **gradient ascent** instead of gradient descent.

$$\theta_{t+1} = \theta_{t} + \alpha \nabla J(\theta_{t})$$
>*\"When the solution is simple, God is answering.\" - Albert Einstein*

We're going to use gradient ascent to maximize our expected reward, but before we can do that we need to define the following derivative $\nabla J(\theta_{t})$.
Now we can ask you to derive this, but we're going to save you some time and give you the answer:
$$\nabla J(\theta_{t}) = \nabla_{\theta} \sum_{s}\mu(s)\sum_{a}q_{\pi}(s, a)\pi_{\theta}(a|s,\theta)$$
$$\nabla J(\theta_{t}) \propto \sum_{s}\mu(s)\sum_{a}q_{\pi}(s, a)\nabla_{\theta}\pi_{\theta}(a|s,\theta)$$
We can prove this mathematicallly, but for now you're going to have to trust us. \
Now that we have defined our gradient update, let's take a look at an example to solidify our understanding.

### 1.2.1 Example

Lets consider an MDP with a single state $s$ and three actions $a_{1}, a_{2}, a{_3}$. Lets assume we start off with an approximate q_value function $q(s, a)$.\
Since we have only one state we can ignore the $\sum_{s}\mu(s)$ term.\
Let's use the *softmax in action preferences* policy:
\begin{equation*}
\pi(a|s, \theta) = \frac{e^{h(s, a, \theta)}} {\sum_{b}{e^{h(s, b, \theta)}}}
\end{equation*}
So out gradient update will be:

$$\theta_{t+1} = \theta_{t} + \alpha \sum_{a}q_{\pi}(s, a)\nabla_{\theta}\pi_{\theta}(a|s,\theta)$$
We have $q(s, a1) = 10, q(s, a2) = 5, q(s, a3) = 2.5$\
Let's assume that $a1$ is our optimal action in state $s$. A

In [48]:
#Read a little more and explain points about the theorem - what u(s) is etc
#Basically try and explain the equation

In [ ]:
#<img src="pg-def.png" />

In [47]:
np.random.randn()

-0.7224578170133801

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable as V

import gym
import gym.spaces
import random
gym.logger.set_level(gym.logger.ERROR)
import numpy as np
from collections import namedtuple
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time
import matplotlib.pyplot as plt
from matplotlib import animation
from builtins import super
from IPython.display import display, Image

In [21]:
a = widgets.FloatSlider(min=0.0, max=1.0, value=1.0, step=0.010)
b = widgets.FloatSlider(min=0.0, max=1.0, value=1.0, step=0.005)
c = widgets.FloatSlider(min=0.0, max=1.0, value=1.0, step=0.0025)

In [22]:
sliders = [a, b, c]

In [36]:
sliders

[FloatSlider(value=1.0, max=1.0, step=0.01),
 FloatSlider(value=1.0, max=1.0, step=0.005),
 FloatSlider(value=1.0, max=1.0, step=0.0025)]

In [26]:
init=[0.0]*3

In [29]:
for i, s in zip(init, sliders):
    print(s)
    s.logit = i
    s.q_val = s.step * 1000

FloatSlider(value=1.0, max=1.0, step=0.01)
FloatSlider(value=1.0, max=1.0, step=0.005)
FloatSlider(value=1.0, max=1.0, step=0.0025)


In [33]:
sliders[0].q_val

10.0

In [37]:
sliders[1].logit

0.0

In [ ]:
lr = 0.5

def update_known_optimal(actions):
    # Each action has a .logit attribute that we must update. It also has a .grad attribute
    # that gives the gradient of the that action.
    actions[0].logit += lr * actions[0].grad
    # We return the index of the action we updated and the size of the arrow to draw
    # (which should be the size of our update, divided by a scaling factor to make the 
    # visual more pleasant)
    return 0, lr / 5

In [ ]:
for i, s in zip(init, sliders):
        s.logit = i
        s.q_val = s.step * 1000
    
    def update_values():
        exps = [np.exp(e.logit) for e in sliders]
        for ex, slid in zip(exps, sliders):
            slid.value = ex / np.sum(exps)
            slid.grad = slid.value * (1-slid.value)
            
    update_values()
    
    def f(a_val10, b_val8, c_val7):
        pass
    
    np.random.seed(1)
    it = 0
    animation_data = []
    while all([v.value < 0.95 for v in [a, b, c]]):
        incr_idx, incr_size = update_fn(sliders)
        update_values()
        it += 1
        animation_data.append((incr_idx, incr_size, [v.value for v in sliders]))
        
    if filename is not None:
        fig, ax = plt.subplots()

        def plot_animation(i):
            plt.clf()
            incr_idx, incr_size, values = animation_data[i]
            plt.bar(['Val = %s' % (v.step * 1000) for v in sliders], values, width=0.5)
            plt.ylim(0, 1.19)
            if incr_size > 0:
                plt.annotate('',
                    xy=(incr_idx, values[incr_idx] + incr_size + 0.05), xycoords='data',
                    xytext=(incr_idx, values[incr_idx]), textcoords='data',
                    arrowprops=dict(width=5, connectionstyle="arc3", color='green'),
                )
            else:
                plt.annotate('',
                    xy=(incr_idx, values[incr_idx]), xycoords='data',
                    xytext=(incr_idx, values[incr_idx] - incr_size + 0.05), textcoords='data',
                    arrowprops=dict(width=5, connectionstyle="arc3", color='red'),
                )
            return fig,

        ani = animation.FuncAnimation(fig, plot_animation, frames=list(range(0, len(animation_data))), blit=False)
        ani.save(filename, writer='imagemagick', fps=10)
        plt.close()
        display(Image(filename)) 

In [ ]:
It is natural to expect policy-based methods are more useful in the continuous space. Because there is an infinite number of actions and (or) states to estimate the values for and hence value-based approaches are way too expensive computationally in the continuous space. For example, in generalized policy iteration, the policy improvement step argmaxa∈Qπ(s,a)
arg
⁡
max
a
∈
A
Q
π
(
s
,
a
)
 requires a full scan of the action space, suffering from the curse of dimensionality.

Using gradient ascent, we can move θ
θ
 toward the direction suggested by the gradient ∇θJ(θ)
∇
θ
J
(
θ
)
 to find the best θ
θ
 for πθ
π
θ
 that produces the highest return.